In [ ]:
!pip install hopsworks

In [ ]:
import pandas as pd
import hopsworks
import joblib

project = hopsworks.login()
fs = project.get_feature_store()

## Load the Model from Model Registry

In [54]:
mr = project.get_model_registry()
model = mr.get_model("car_prices", version=1)
model_dir = model.download()
model = joblib.load(model_dir + "/car_prices_model.pkl")

Connected. Call `.close()` to terminate connection gracefully.

In [55]:
feature_view = fs.get_feature_view(name="car_prices", version=1)

In [56]:
import datetime
from PIL import Image

batch_data = feature_view.get_batch_data()

features_df = batch_data.drop('full_name', axis = 1)

predictions = model.predict(features_df)

In [57]:
car = batch_data.iloc[[predictions.size-1]]

In [58]:
import pandas as pd

monitor_fg = fs.get_or_create_feature_group(name="car_predictions",
                                  version=1,
                                  primary_key=["datetime"],
                                  description="Car Price Prediction/Outcome Monitoring"
                                 )

In [59]:
from datetime import datetime
now = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")

data = {
    'prediction': predictions[-1],
    'label': car.iloc[0][0],
    'datetime': [now],
}
monitor_df = pd.DataFrame(data)
monitor_fg.insert(monitor_df)

Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3342/jobs/named/car_predictions_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f6124763e90>, None)

In [61]:
history_df = monitor_fg.read()
history_df

,prediction,label,datetime
0,42.7945,Audi Q5 3.0 TDI Quattro Technology,"11/12/2022, 14:32:15"
1,9.3547,Ford Endeavour Hurricane Limited Edition,"11/12/2022, 15:57:20"


In [47]:
! pip install dataframe-image --quiet

     |████████████████████████████████| 6.6 MB 5.1 MB/s 


In [62]:
import dataframe_image as dfi

df_recent = history_df.tail(5)
 
dfi.export(df_recent, 'assets/df_recent.png', table_conversion = 'matplotlib')

In [67]:
import datetime
start_date = (datetime.datetime.now() - datetime.timedelta(hours=240)) 
end_date = (datetime.datetime.now()) 
print(start_date)
print(end_date)

2022-11-02 16:03:51.083559
2022-11-12 16:03:51.083650


In [68]:
batch_data['prediction'] = predictions.tolist()
results_df = batch_data.copy()[["full_name",  "prediction"]]

In [69]:
results_df.loc[:,'batch_start_date'] = start_date
results_df

,full_name,prediction,batch_start_date
0,Maruti Vitara Brezza VDi Option,7.487500,2022-11-02 16:03:51.083559
1,Maruti Swift Dzire ZXI 1.2 BS IV,6.457975,2022-11-02 16:03:51.083559
2,Ford Ecosport 2015-2021 1.5 Diesel Trend Plus ...,7.962400,2022-11-02 16:03:51.083559
3,Hyundai Grand i10 CRDi Asta,6.511600,2022-11-02 16:03:51.083559
4,Mahindra KUV 100 mFALCON G80 K8,6.855000,2022-11-02 16:03:51.083559
...,...,...,...
9694,Ford Endeavour 3.2 Titanium AT 4X4,28.432500,2022-11-02 16:03:51.083559
9695,Honda City i-VTEC VX,10.326600,2022-11-02 16:03:51.083559
9696,Hyundai Elantra CRDi SX,6.041300,2022-11-02 16:03:51.083559
9697,Audi Q5 3.0 TDI Quattro Technology,41.344900,2022-11-02 16:03:51.083559


In [70]:
results_fg = fs.get_or_create_feature_group(
    name="car_price_results",
    version=1,
    description="Predicted prices of cars",
    primary_key=["full_name"]
)
results_fg.insert(results_df)

Uploading Dataframe: 0.00% |          | Rows 0/9699 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3342/jobs/named/car_price_results_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f611f6f0090>, None)